## Data Exploration

This is where we're going to inspect the dataset we've been downloaded.

<p style='color: yellow;'><i>Note: You need to run on the download_data.py at the scripts folder before to use this notebook</i></p>

#### Getting e-commerce data

In [16]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [17]:
def smart_csv_conversion():
    """
    Smart CSV conversion with duplication prevention and file integrity checks
    """
    excel_path = '../data/online_retail.xlsx'
    csv_path = '../data/online_retail.csv'
    
    # Check if files exist
    excel_exists = os.path.exists(excel_path)
    csv_exists = os.path.exists(csv_path)
    
    if not excel_exists:
        raise FileNotFoundError(f"❌ Excel file not found: {excel_path}")
    
    # Get file modification times for comparison
    excel_mtime = os.path.getmtime(excel_path)
    csv_mtime = os.path.getmtime(csv_path) if csv_exists else 0
    
    # Check file sizes to detect duplicates/corruption
    excel_size = os.path.getsize(excel_path)
    csv_size = os.path.getsize(csv_path) if csv_exists else 0
    
    print(f"📊 Excel file: {excel_size / 1024**2:.2f} MB")
    if csv_exists:
        print(f"📊 CSV file: {csv_size / 1024**2:.2f} MB")
    
    # Decision logic
    should_convert = False
    reason = ""
    
    if not csv_exists:
        should_convert = True
        reason = "CSV file doesn't exist"
    elif excel_mtime > csv_mtime:
        should_convert = True
        reason = "Excel file is newer than CSV"
    elif csv_size == 0:
        should_convert = True
        reason = "CSV file is empty (corrupted)"
        print("⚠️ Detected empty CSV file - removing corrupted file")
        os.remove(csv_path)
    elif csv_size < 1000:  # Suspiciously small CSV
        should_convert = True
        reason = "CSV file is suspiciously small (likely corrupted)"
        print("⚠️ Detected corrupted CSV file - removing")
        os.remove(csv_path)
    else:
        # Additional integrity check - try to read a few rows
        try:
            test_df = pd.read_csv(csv_path, nrows=5)
            if len(test_df) == 0 or len(test_df.columns) < 3:
                should_convert = True
                reason = "CSV file appears corrupted (no data or too few columns)"
                print("⚠️ CSV integrity check failed - removing corrupted file")
                os.remove(csv_path)
            else:
                print("✅ CSV file exists and appears valid")
                print(f"📅 CSV last modified: {datetime.fromtimestamp(csv_mtime)}")
                print(f"📅 Excel last modified: {datetime.fromtimestamp(excel_mtime)}")
        except Exception as e:
            should_convert = True
            reason = f"CSV file is corrupted: {str(e)}"
            print(f"⚠️ CSV read error: {e} - removing corrupted file")
            if os.path.exists(csv_path):
                os.remove(csv_path)
    
    # Perform conversion if needed
    if should_convert:
        print(f"🔄 Converting Excel to CSV - Reason: {reason}")
        try:
            # Read Excel file
            print("📖 Reading Excel file...")
            df = pd.read_excel(excel_path)
            
            # Validation before saving
            if df.empty:
                raise ValueError("Excel file is empty!")
            
            if len(df.columns) < 3:
                raise ValueError("Excel file has too few columns!")
            
            # Save to CSV
            print("💾 Saving to CSV...")
            df.to_csv(csv_path, index=False)
            
            # Verify the saved file
            verification_df = pd.read_csv(csv_path, nrows=5)
            if len(verification_df) == 0:
                raise ValueError("CSV verification failed!")
            
            print(f"✅ Successfully converted to CSV")
            print(f"📊 Rows: {len(df):,}, Columns: {len(df.columns)}")
            print(f"💾 CSV size: {os.path.getsize(csv_path) / 1024**2:.2f} MB")
            
            return df
            
        except Exception as e:
            print(f"❌ Conversion failed: {e}")
            # Clean up failed CSV file
            if os.path.exists(csv_path):
                os.remove(csv_path)
            raise
    else:
        print("⚡ Using existing CSV file")
        return pd.read_csv(csv_path)

def load_data_smart():
    """
    Smart data loading with automatic CSV conversion and integrity checks
    """
    print("🚀 Starting smart data loading...")
    
    try:
        df = smart_csv_conversion()
        
        # Load the CSV (either existing or newly created)
        if not isinstance(df, pd.DataFrame):
            print("📖 Loading CSV file...")
            df = pd.read_csv('../data/online_retail.csv')
        
        # Final validation
        print(f"\n📊 Final Dataset Info:")
        print(f"   Shape: {df.shape}")
        print(f"   Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        print(f"   Columns: {list(df.columns)}")
        
        return df
        
    except Exception as e:
        print(f"❌ Loading failed: {e}")
        print("🔄 Falling back to direct Excel loading...")
        return pd.read_excel('../data/online_retail.xlsx')

# Execute smart loading
df = load_data_smart()

# Display basic info
print("\n👀 First 5 rows:")
print(df.head())

# BONUS: Check for potential duplicates in the dataset itself
def check_data_duplicates(df):
    """Check for duplicate rows in the dataset"""
    duplicates = df.duplicated().sum()
    if duplicates > 0:
        print(f"⚠️ Found {duplicates:,} duplicate rows in dataset")
        print("🧹 Consider removing duplicates with: df.drop_duplicates()")
    else:
        print("✅ No duplicate rows found in dataset")

print("\n🔍 Checking for data duplicates...")
check_data_duplicates(df)

🚀 Starting smart data loading...
📊 Excel file: 22.62 MB
📊 CSV file: 46.33 MB
✅ CSV file exists and appears valid
📅 CSV last modified: 2025-06-01 19:08:34.307202
📅 Excel last modified: 2025-05-31 15:44:20.704262
⚡ Using existing CSV file

📊 Final Dataset Info:
   Shape: (541909, 8)
   Memory: 175.20 MB
   Columns: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']

👀 First 5 rows:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

           InvoiceDate  UnitPrice  CustomerID         Country  
0  2010-12-01 08:26:00       2.55     17850.0  United Kingdom  


### Data Quality Assessment

In [18]:
# Check for missing values
print("Missing Values:")
print(df.isnull().sum())

# Data types
print("\nData Types:")
print(df.dtypes)

# Basic statistics
print("\nBasic Statistics:")
df.describe()

Missing Values:
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Data Types:
InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

Basic Statistics:


,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [19]:
# Save DataFrame for next notebook
df.to_pickle('../data/processed_df.pkl')
print("✅ DataFrame saved for next notebook!")

✅ DataFrame saved for next notebook!


In [ ]:
# Remove negative quantities (returns) for basic analysis
df_clean = df_clean[df_clean['Quantity'] > 0]
df_clean = df_clean[df_clean['UnitPrice'] > 0]

# Remove extreme outliers
Q1 = df_clean['TotalAmount'].quantile(0.25)
Q3 = df_clean['TotalAmount'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_clean = df_clean[(df_clean['TotalAmount'] >= lower_bound) & 
                   (df_clean['TotalAmount'] <= upper_bound)]